In [ ]:
#| default_exp math

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from scipy.special import factorial
import itertools as it

from fastcore.test import *

In [ ]:
#| export
def comb(n, k):
    """Vectorized combination: `comb(n,k)` = n! / ((n-k)!k!)
    
    Parameters
    ----------
    n : int or np.array of int
        First parameter of combination
    k : int or np.array of int
        Second parameter of combination
        
    Returns
    -------
    np.array
        Combination (choose k out of n)
    """
    return factorial(n) / (factorial(k) * factorial(n-k))

In [ ]:
#| export
def binom(k, n, p):
    """Vectorized binomial distribution: `binom(k,n,p)`=`comb(n,k)` p^k (1-p)^n-k
    
    Example
    -------
    >> binom(k=[1,2], n=[3,4], p=0.1)
    [0.243 , 0.0486]
    
    Parameters
    ----------
    n : int or list of int
        First parameter of combination
    k : int or list of int
        Second parameter of combination
    p : float or list of float
        Probability 
    
    Returns
    -------
    np.array
        Value(s) of binomial distribution evaluated at k,n,p.
    """
    k, n, p = np.array(k), np.array(n), np.array(p)
    return comb(n,k) * p**k * (1-p)**(n-k)

In [ ]:
test_close(binom(k=[1,2], n=[3,4], p=0.1), [0.243 , 0.0486], eps=1e-05)

In [ ]:
#| export
def joint_binom(k, n, p):
    """Product of independent binomial distributions with
    parameters `k`, `n` and `p` (can be list of lists), i.e.:
    
    `joint_binom(k,n,p)`=`binom(k[0],n[0],p[0])`×...×`binom(k[-1],n[-1],p[-1])`
    
    Example
    -------
    >> joint_binom(k=[1,1], n=[2,2], p=[0.5,0.5])
    0.25
    
    Parameters
    ----------
    n : list of int
        List of first parameters of combination
    k : list of int
        List of second parameters of combination
    p : list of list, list of float, or float
        Probability
        
    Returns
    -------
    np.array
        Joint probability
    """
    return np.prod(binom(k,n,p), axis=-1) # In case p is list of list: vector, else scalar

In [ ]:
test_close(joint_binom(k=[1,2], n=[3,4], p=[0.1,0.2]), binom(k=1,n=3,p=0.1) * binom(k=2,n=4,p=0.2))
test_close(joint_binom(k=[1,2], n=[2,3], p=[[0.1,0.2],[.3,.4]]), [binom(1,2,0.1)*binom(2,3,0.2), binom(1,2,0.3)*binom(2,3,0.4)])

In [ ]:
#| export
def Wilson_var(p, N):
    """Wilson estimator of binomial variance (see Eq. C12 in paper)
    
    The formula for the Wilson interval is:
    
        CI = p+z^2/(2n) \pm z\sqrt{pq/n + z^2/(4n^2)}/(1 + z^2/n)
    
    we can extract the var (z=1) as:
    
        Var(p) = (CI/2)^2 = (npq + 0.25) / (1 + n)^2
    
    Parameters
    ----------
    p : float
        Estimator of probability
    N : int
        Sample size
        
    Returns
    -------
    float
        Estimated variance of Wilson CI
    """
    return (N*p*(1-p) + 0.25)  / (N**2 + 2*N + 1)

In [ ]:
#| export
def Wald_var(p, N):
    """Wald estimation of binomial variance (see Eq. C11 in paper)
    
    Parameters
    ----------
    p : float
        Estimator of probability
    N : int
        Sample size
        
    Returns
    -------
    float
        Estimated variance of Wald CI
    """
    return p * (1-p) / N

In [ ]:
#| export
def subset_cards(superset):
    """Calculate cardinalities of all possible subsets of `superset`
    
    Example
    -------
    >> subset_cards({1,2,3})
    {0,1,2,3}
    
    Parameters
    ----------
    superset : set
        Input set
    
    Returns
    -------
    list of int
        All possible cardinalities of subsets in superset
    """
    return set(range(len(superset) + 1))

In [ ]:
assert(subset_cards({1,2,3}) == {0,1,2,3})
assert(subset_cards({(0,0), (0,1), (0,2)}) == {0,1,2,3})

In [ ]:
#| export
def cartesian_product(list_of_sets):
    """Calculate cartesian product between all members of sets
    
    Example
    -------
    >> cartesian_product([{1,2}, {3,4}])
    [(1,3), (1,4), (2,3), (2,4)]
    
    Parameters
    ----------
    list_of_sets : list
        List of sets between which to calculate Cartesian product
        
    Returns
    -------
    list of tuple
        Cartesian products
    """
    return list(it.product(*list_of_sets))

In [ ]:
assert(cartesian_product([{1,2},{3,4}]) == [(1,3), (1,4), (2,3), (2,4)])

In [ ]:
#| export
def subset_probs(circuit, error_model, prob):
    """Calculate occurence probability of subsets in `circuit` with physical
    error rate `prob`. `error_model` defines how the circuit is to be 
    partitioned before occurence probabilities are calculated. (see Eq. 2 in paper)
    
    Example
    -------
    >> subset_probs(qsample.examples.por, qsample.noise.E1, 0.1)
    {(0,): 0.6561,
     (1,): 0.2916,
     (2,): 0.04860000000000001,
     (3,): 0.0036000000000000008,
     (4,): 0.00010000000000000002}
    
    Parameters
    ----------
    circuit : Circuit
        Circuit wrt. which subset probabilities are calculated
    error_model : ErrorModel
        Error model by which to partition `circuit`
    prob : float or list of float
        Physical error probabilities 
    Returns
    -------
    dict
        keys: subset, values: corresponding probability
    """
    list_of_sets = error_model.group(circuit).values()
    cards = list(map(len, list_of_sets))
    cp_subset_cards = cartesian_product([subset_cards(s) for s in list_of_sets])
    return {cp : joint_binom(cp, cards, prob) for cp in cp_subset_cards}